#  The Moving Average Crossover Strategy (Long-short)

We have two SMA filters viz. the shorter lookback period SMA (henceforth referred to as 'SMA50') and the longer lookback period SMA (henceforth referred to as 'SMA200'). We go long on Tata Steel at the first instance when the SMA50 exceeds the SMA200. Similarly, we go short on it, at the first instance when the SMA200 exceeds the SMA50.

Our trading rules can be stated as

Go long when SMA50 > SMA200 on a given day and SMA50 < SMA200 on the previous day
Go short when SMA50 < SMA200 on a given day and SMA50 > SMA200 on the previous day

Import the required libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import warnings 

warnings.filterwarnings('ignore')

Fix the backtesting period

In [2]:
# Backtesting period should be of 5 years
end = pd.datetime.now().date()
start = end - pd.Timedelta(days=10*252)

Create a class and create instance attributes/methods which are required for backtesting

In [3]:
# Create a class
class backtesting_Moving_Average_Cross_Over:
    
    # Create Instance attribute and methods
    def __init__(self,ticker,start,end,long_sma,short_sma):
        
        self.ticker = ticker
        self.start = start
        self.end = end
        self.long_sma = long_sma
        self.short_sma = short_sma 
        
        self.fetch_data()
        self.indicators()
        self.signals()
        self.positions()
        self.returns()
        
    #To fetch the required data from yahoo finance    
    def fetch_data(self):
        self.df = yf.download(self.ticker,self.start,self.end)
    
    # To compute the required indicators Long SMA and Short SMA
    def indicators(self):
        self.df['long_sma'] = self.df['Adj Close'].rolling(window=self.long_sma).mean()
        self.df['short_sma'] = self.df['Adj Close'].rolling(window=self.short_sma).mean()
        
    # To generate the long and short entry signal
    def signals(self):
        self.df['signals'] = np.where((self.df['short_sma'].shift() < self.df['long_sma'].shift()) & (self.df['short_sma'] > self.df['long_sma']),1,0)
        self.df['signals'] = np.where((self.df['short_sma'].shift() > self.df['long_sma'].shift()) & (self.df['short_sma'] < self.df['long_sma']),-1,self.df['signals'])
        
    # To mark the positions taken
    def positions(self):
        self.df['positions'] = self.df['signals'].shift()
        #Positions taken all the way whenever the long and short crosses
        self.df['positions'] = self.df['positions'].replace(to_replace=0,method='ffill')
    
    #To compute the returns generated
    def returns(self):
        self.df['BnH Returns'] = np.log(self.df['Adj Close']/self.df['Adj Close'].shift(1))
        self.df['Strategy Returns'] = self.df['positions'] * self.df['BnH Returns']
        # BnH returns is the returns generated if bought and hold it for the mentioned period simply without any strategy
        # Strategy Returns is the returns generated due to implementation of this strategy on this historical data
        Strat_Returns = self.df['Strategy Returns'].cumsum()[-1]
        BnH_Returns = self.df['BnH Returns'].cumsum()[-1]
        print('Buy and Hold Returns :', self.df['BnH Returns'].cumsum()[-1])
        print('Strategy Returns :', self.df['Strategy Returns'].cumsum()[-1])
        return Strat_Returns, BnH_Returns
    

In [4]:
# Try with SPY index by passing relative range as 25 and internal bar strength as 0.3, this will return the buy and hold return & strategy return 
TataSteel_backtesting_MAC = backtesting_Moving_Average_Cross_Over('TATASTEEL.NS',start,end,200,50)

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.3795073253749854
Strategy Returns : 0.9170667083469828


Optimise the strategy by considering few other stocks with the various Long and Short term SMA ranges

In [5]:
long_sma_list=[]
short_sma_list=[]
Returns=[]
stocks=[]

stocks_list = ['BHARTIARTL.NS','NTPC.NS','MARUTI.NS','HINDALCO.NS','ONGC.NS','KOTAKBANK.NS','BAJAJ-AUTO.NS','COALINDIA.NS',
                      'ITC.NS','ULTRACEMCO.NS','CIPLA.NS','LT.NS','INDUSINDBK.NS','HDFCLIFE.NS','TATACONSUM.NS','RELIANCE.NS',
                      'TATASTEEL.NS','ICICIBANK.NS','GRASIM.NS','NESTLEIND.NS','TCS.NS','TECHM.NS','SHREECEM.NS','BAJFINANCE.NS',
                      'BAJAJFINSV.NS','WIPRO.NS','BRITANNIA.NS','HEROMOTOCO.NS','TITAN.NS']

for i in range(30,60,10):
    for j in range(150,250,25):
        for k in stocks_list:
            a = backtesting_Moving_Average_Cross_Over(k,start,end,j,i)
            stocks.append(k)
            Returns.append(a.returns())
            long_sma_list.append(j)
            short_sma_list.append(i)
            
Strat = [x[0] for x in Returns]
BnH = [x[1] for x in Returns]
    

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.8559642881239325
Strategy Returns : 0.4067521532973547
Buy and Hold Returns : 0.8559642881239325
Strategy Returns : 0.4067521532973547
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.39719978707484693
Strategy Returns : -0.2219916947072338
Buy and Hold Returns : 0.39719978707484693
Strategy Returns : -0.2219916947072338
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.915161295749944
Strategy Returns : 0.5307619209416157
Buy and Hold Returns : 0.915161295749944
Strategy Returns : 0.5307619209416157
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.2963608852801998
Strategy Returns : 2.2259855329941396
Buy and Hold Returns : 1.2963608852801998
Strategy Returns : 2.2259855329941396
[*********************100%***********************]  1 of 1 completed
Buy and Hold 

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : -0.2673823301398814
Strategy Returns : -0.14806694570120257
Buy and Hold Returns : -0.2673823301398814
Strategy Returns : -0.14806694570120257
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.1433152113853021
Strategy Returns : -0.07561894703626722
Buy and Hold Returns : 0.1433152113853021
Strategy Returns : -0.07561894703626722
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.8624884914931598
Strategy Returns : -0.8294416840368989
Buy and Hold Returns : 0.8624884914931598
Strategy Returns : -0.8294416840368989
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.2673447609848384
Strategy Returns : -0.3149813883321242
Buy and Hold Returns : 0.2673447609848384
Strategy Returns : -0.3149813883321242
[*********************100%***********************]  1 of 1 completed
B

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.602856530143819
Strategy Returns : 1.0622244727074415
Buy and Hold Returns : 1.602856530143819
Strategy Returns : 1.0622244727074415
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.738438296473713
Strategy Returns : 0.2822325379372649
Buy and Hold Returns : 1.738438296473713
Strategy Returns : 0.2822325379372649
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.379507437157243
Strategy Returns : 1.1227380350329526
Buy and Hold Returns : 1.379507437157243
Strategy Returns : 1.1227380350329526
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.017159541633847
Strategy Returns : 0.2305147059918728
Buy and Hold Returns : 1.017159541633847
Strategy Returns : 0.2305147059918728
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.8645927799040011
Strategy Returns : 0.1428741326583937
Buy and Hold Returns : 0.8645927799040011
Strategy Returns : 0.1428741326583937
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.8025341201343555
Strategy Returns : -0.9344161261974624
Buy and Hold Returns : 0.8025341201343555
Strategy Returns : -0.9344161261974624
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 2.8077302724182465
Strategy Returns : 0.7805406131688248
Buy and Hold Returns : 2.8077302724182465
Strategy Returns : 0.7805406131688248
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 2.3393657776880024
Strategy Returns : 0.5462850525775025
Buy and Hold Returns : 2.3393657776880024
Strategy Returns : 0.5462850525775025
[*********************100%***********************]  1 of 1 completed
Buy and Hold 

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.7980399717376847
Strategy Returns : -0.08555063400609325
Buy and Hold Returns : 1.7980399717376847
Strategy Returns : -0.08555063400609325
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.8559644888154362
Strategy Returns : 0.2834671917635726
Buy and Hold Returns : 0.8559644888154362
Strategy Returns : 0.2834671917635726
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.39719986805875385
Strategy Returns : -0.004223439418344292
Buy and Hold Returns : 0.39719986805875385
Strategy Returns : -0.004223439418344292
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.9151612957499389
Strategy Returns : -0.16756687520186558
Buy and Hold Returns : 0.9151612957499389
Strategy Returns : -0.16756687520186558
[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.689510470960111
Strategy Returns : -0.6530865364622032
Buy and Hold Returns : 0.689510470960111
Strategy Returns : -0.6530865364622032
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : -0.2673824023871942
Strategy Returns : -0.0916930102609345
Buy and Hold Returns : -0.2673824023871942
Strategy Returns : -0.0916930102609345
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.14331504952806276
Strategy Returns : -0.30491297113488436
Buy and Hold Returns : 0.14331504952806276
Strategy Returns : -0.30491297113488436
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.8624884914931611
Strategy Returns : -0.9824143847660335
Buy and Hold Returns : 0.8624884914931611
Strategy Returns : -0.9824143847660335
[*********************100%***********************]  1 of 1 completed
Buy

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.5990144332387808
Strategy Returns : -0.2216182515342143
Buy and Hold Returns : 0.5990144332387808
Strategy Returns : -0.2216182515342143
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.6028565301438178
Strategy Returns : 1.3129158468232458
Buy and Hold Returns : 1.6028565301438178
Strategy Returns : 1.3129158468232458
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.7384382964737108
Strategy Returns : -0.827971790050779
Buy and Hold Returns : 1.7384382964737108
Strategy Returns : -0.827971790050779
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.3795074371572449
Strategy Returns : 1.2894917735651035
Buy and Hold Returns : 1.3795074371572449
Strategy Returns : 1.2894917735651035
[*********************100%***********************]  1 of 1 completed
Buy and Hold 

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.1595358228104566
Strategy Returns : 0.24292728420243137
Buy and Hold Returns : 1.1595358228104566
Strategy Returns : 0.24292728420243137
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.864592882622505
Strategy Returns : -0.2251893534399053
Buy and Hold Returns : 0.864592882622505
Strategy Returns : -0.2251893534399053
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.8025343910883692
Strategy Returns : -1.4008189194966052
Buy and Hold Returns : 0.8025343910883692
Strategy Returns : -1.4008189194966052
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 2.8077301987722807
Strategy Returns : 0.32945064769354954
Buy and Hold Returns : 2.8077301987722807
Strategy Returns : 0.32945064769354954
[*********************100%***********************]  1 of 1 completed
Buy and H

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.21060644377646598
Strategy Returns : 0.0064520571165385746
Buy and Hold Returns : 0.21060644377646598
Strategy Returns : 0.0064520571165385746
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.7980399717376807
Strategy Returns : 0.10350049150787408
Buy and Hold Returns : 1.7980399717376807
Strategy Returns : 0.10350049150787408
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.8559642881239357
Strategy Returns : 0.26590016734509914
Buy and Hold Returns : 0.8559642881239357
Strategy Returns : 0.26590016734509914
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.39719962510707085
Strategy Returns : 0.14629338667127648
Buy and Hold Returns : 0.39719962510707085
Strategy Returns : 0.14629338667127648
[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.0267567984838366
Strategy Returns : -0.5824139606075008
Buy and Hold Returns : 1.0267567984838366
Strategy Returns : -0.5824139606075008
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.6895104709601112
Strategy Returns : -0.9530739463955242
Buy and Hold Returns : 0.6895104709601112
Strategy Returns : -0.9530739463955242
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : -0.26738233013988005
Strategy Returns : 0.023732965022950716
Buy and Hold Returns : -0.26738233013988005
Strategy Returns : 0.023732965022950716
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.1433150495280596
Strategy Returns : 0.4097159604891264
Buy and Hold Returns : 0.1433150495280596
Strategy Returns : 0.4097159604891264
[*********************100%***********************]  1 of 1 completed
Buy

In [6]:
# Create a dataframe and validate the returns generated for various indices with different EMA range
Results = pd.DataFrame({'Stocks':stocks,'Short SMA':short_sma_list,'Long SMA':long_sma_list,'Strategy Returns':Strat,'BnH Returns':BnH})


In [7]:
# Shows the record in descending order which has maximum return 
Results.sort_values(by='Strategy Returns',inplace=True,ascending=False)

Note : This is simple strategy to learn how to backtest and optimize the parameters by reusing the code. This can be enhanced by coding the performance metrics such as max drawdown, sharpe ratio, hit ratio e.t.c.

In [8]:
Results.head(20)

,Stocks,Short SMA,Long SMA,Strategy Returns,BnH Returns
3,HINDALCO.NS,30,150,2.225986,1.296361
119,HINDALCO.NS,40,150,1.843099,1.296361
32,HINDALCO.NS,30,175,1.663244,1.296361
103,TATASTEEL.NS,30,225,1.549800,1.379508
61,HINDALCO.NS,30,200,1.448083,1.296361
148,HINDALCO.NS,40,175,1.421691,1.296361
226,BAJFINANCE.NS,40,225,1.359048,2.807730
333,TATACONSUM.NS,50,225,1.346464,1.602856
101,TATACONSUM.NS,30,225,1.323914,1.602857
217,TATACONSUM.NS,40,225,1.312916,1.602857
